In [1]:
import nest_asyncio
from dotenv import load_dotenv
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# with unstructured

from unstructured_client import UnstructuredClient
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError
import os
import time

from utils.markdown_utils import unstructured_elements_to_markdown


In [2]:
nest_asyncio.apply()
load_dotenv()


True

In [3]:
out_path = "../data/out/"
origin_path = "../data/pdf/"

# Get all the filenames from the origin path
file_names = os.listdir(origin_path)
file_names.sort()
file_names = [{"path": name} for name in file_names]

print("len:", len(file_names))


len: 329


## Converting PDF data to TXT


In [4]:
s = UnstructuredClient(
    api_key_auth=os.environ["UNSTRUCTURED_API_KEY"],
    server_url=os.environ["UNSTRUCTURED_SERVER_URL"],
)


In [5]:
def parse_pdf_to_txt(file, idx):
    file_path = origin_path + file["path"]
    print("Processing file:", file_path)
    with open(file_path, "rb") as f:
        # Note that this currently only supports a single file
        files = shared.Files(
            content=f.read(),
            file_name=file_path,
        )

    req = shared.PartitionParameters(
        files=files,
        # Other partition params
        strategy="fast",
        languages=["eng"],
        # split_pdf_allow_failed=True
        # pdf_infer_table_structure=True,
        # skip_infer_table_types=[],
    )

    try:
        resp = s.general.partition(req)
        # print(len(resp.elements))
    except SDKError as e:
        print(e)
        return True
    except Exception as e:  # if the SDKError is not caught
        print("Another exception", e)
        return True
    simple_md = unstructured_elements_to_markdown(resp.elements)

    # get the size of the file
    file["size"] = len(simple_md)
    # file_names[idx] = {"path": file["path"], "size": len(simple_md)}

    file_out = out_path + f"txt/" + file["path"].replace(".pdf", ".txt")
    # file_out = file_out.replace("pdf", "txt")

    with open(file_out, "w") as f:
        f.write(simple_md)

    return False


In [ ]:
for idx, file in enumerate(file_names):
    with_error = parse_pdf_to_txt(file, idx)
    try_limit = 3
    while with_error and try_limit > 0:
        with_error = parse_pdf_to_txt(file, idx)
        try_limit -= 1
        time.sleep(4)

    if try_limit == 0:
        print("Error processing file:", file["path"])
        file["unstructured_error"] = True

    time.sleep(2)


In [6]:
# process the files with errors
file_names = [file for file in file_names if "unstructured_error" in file]
print(file_names)


[]


Manual verification the files with smallest size


IMPORTANT: The next piece of code is unnecessary, was writed just because the original file_names war empty after running the last function (probably a bug in the function or by ram memory).


In [15]:
# get the files names from the out path
file_names_out = os.listdir(out_path + "txt/")
file_names_out.sort()

file_names_out = [{"path": name} for name in file_names_out]

# get the size of each file
for idx, file in enumerate(file_names_out):
    file_path = out_path + "txt/" + file["path"]
    with open(file_path, "r") as f:
        file["size"] = len(f.read())

# print the files with size less than 300

import json

if len(file_names) == 0:
    file_names = [file for file in file_names_out]
    for file in file_names:
        file["path"] = file["path"].replace(".txt", ".pdf")

file_names_out = [file for file in file_names_out if file["size"] < 500]
print(json.dumps(file_names, indent=4))


[
    {
        "path": "1--ELA-Pilotpdf.pdf",
        "size": 1372
    },
    {
        "path": "10-Accessing-Your-Student-List-in-PATHpdf.pdf",
        "size": 744
    },
    {
        "path": "100-Evaluating-Your-Gatheringpdf.pdf",
        "size": 2854
    },
    {
        "path": "101-Explain-Auto-drop-to-Students-for-PCpdf.pdf",
        "size": 1789
    },
    {
        "path": "102-Explain-Gospel-Resourcespdf.pdf",
        "size": 1520
    },
    {
        "path": "103-Explain-the-Block-Schedulepdf.pdf",
        "size": 1765
    },
    {
        "path": "104-Explaining-BYUPW-to-Potential-Studentspdf.pdf",
        "size": 8854
    },
    {
        "path": "105-FRENCH-CSM-Application-Octpdf.pdf",
        "size": 7752
    },
    {
        "path": "106-Facilitating-a-Virtual-EC-Gatheringpdf.pdf",
        "size": 2273
    },
    {
        "path": "107-Facilitating-an-In-Person-EC-Gatheringpdf.pdf",
        "size": 2162
    },
    {
        "path": "108-Find-Student-Informationpdf.pdf"

In [16]:
print(len(file_names))


329


# Convert the TXT info to Markdown with Llama-Parse


In [8]:
parser = LlamaParse(
    result_type="markdown",  # "markdown" and "text" are available
    parsing_instruction=(
        "Convert the provided text into accurate and well-structured Markdown format, closely resembling the original PDF structure. "
        "Use headers from H1 to H3, with H1 for main titles, H2 for sections, and H3 for subsections. "
        "Detect any bold, large, or all-uppercase text as headers. "
        "Preserve bullet points and numbered lists with proper indentation to reflect nested lists. "
        "if it is not a header, ensure that bold and italic text is properly formatted using double **asterisks** for bold and single *asterisks* for italic"
        "Detect and correctly format blockquotes using the '>' symbol for any quoted text. "
        "When processing text, pay attention to line breaks that may incorrectly join or split words. "
        "Automatically correct common errors, such as wrongly concatenated words or broken lines, to ensure the text reads naturally"
        "If code snippets or technical commands are found, enclose them in triple backticks ``` for proper formatting. "
        "If any tables are detected, parse them as a title (bold header) followed by list items"
        "If you see the same header multiple times, merge them into one."
        "If images contain important text, transcribe only the highlighted or boxed text and ignore general background text. "
        "The final output should be a clean, concise Markdown document closely reflecting the original PDF's intent and structure without adding any extra text."
    ),
)


In [9]:
file_extractor = {".txt": parser}


def parse_txt_to_md(file, idx):
    load_file = out_path + "txt/" + file["path"].replace(".pdf", ".txt")

    documents = SimpleDirectoryReader(
        input_files=[load_file], file_extractor=file_extractor
    ).load_data()

    size = sum([len(doc.text) for doc in documents])

    out_name = f"{out_path}md/{file['path'].replace('.pdf', '.md')}"
    # out_name = file.replace(".txt", ".md")
    with open(out_name, "w") as f:
        for doc in documents:
            f.write(doc.text)
            f.write("\n\n")
        print(out_name, "saved.")

    return size


In [17]:
for idx, file in enumerate(file_names):
    limit_try = 2
    size = parse_txt_to_md(file, idx)

    while size < (file["size"] - 300) and limit_try > 0:
        try:
            time.sleep(2)
            size = parse_txt_to_md(file, idx)
            limit_try -= 1
            break
        except Exception as e:
            limit_try -= 1
            print(f"Error processing file {file} - {limit_try} tries left.")
            time.sleep(2)

    if limit_try == 0:
        print(f"Error processing file {file} - {limit_try} tries left.")
        file["llama_error"] = True


INFO: HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"


Started parsing the file under job_id e62f7adb-c734-431a-9350-d993253cedec


INFO: HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/e62f7adb-c734-431a-9350-d993253cedec "HTTP/1.1 200 OK"
INFO: HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/e62f7adb-c734-431a-9350-d993253cedec "HTTP/1.1 200 OK"
INFO: HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/e62f7adb-c734-431a-9350-d993253cedec "HTTP/1.1 200 OK"
INFO: HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/e62f7adb-c734-431a-9350-d993253cedec "HTTP/1.1 200 OK"
INFO: HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/e62f7adb-c734-431a-9350-d993253cedec "HTTP/1.1 200 OK"
INFO: HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/e62f7adb-c734-431a-9350-d993253cedec/result/markdown "HTTP/1.1 200 OK"


../data/out/md/1--ELA-Pilotpdf.md saved.


INFO: HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"


...

../data/out/md/98-EnglishConnect-Common-Termspdf.md saved.


INFO: HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"


Started parsing the file under job_id 31e1b626-7b86-48bf-bb41-231e9ef98087


INFO: HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/31e1b626-7b86-48bf-bb41-231e9ef98087 "HTTP/1.1 200 OK"
INFO: HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/31e1b626-7b86-48bf-bb41-231e9ef98087 "HTTP/1.1 200 OK"
INFO: HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/31e1b626-7b86-48bf-bb41-231e9ef98087 "HTTP/1.1 200 OK"
INFO: HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/31e1b626-7b86-48bf-bb41-231e9ef98087 "HTTP/1.1 200 OK"
INFO: HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/31e1b626-7b86-48bf-bb41-231e9ef98087 "HTTP/1.1 200 OK"
INFO: HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/31e1b626-7b86-48bf-bb41-231e9ef98087 "HTTP/1.1 200 OK"
INFO: HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/31e1b626-7b86-48bf-bb41-231e9ef98087 "HTTP/1.1 200 OK"
INFO: HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/31e1b626-7b86-48bf-bb41-231e9ef98087/re

../data/out/md/99-Ensign-College-Overviewpdf.md saved.


In [ ]:
# print the files that could not be loadedand has an error attribute
for file in file_names:
    if "error" in file:
        print(file)


Review if there are files with error to review them manually.


In [ ]:
files_with_error = [file for file in file_names if "error" in file]

print(files_with_error)


### Pros

- Cheapest because it makes less requests to the API
- Most accurate because it uses the text directly
- Well structured result

### Cons

- We are ignoring completely the image and only using the text related. This could be a problem if the image contains important information
- Sometimes the order of the text is not the same as the PDF because the structure (i.e. 18)

### warnings

- In some PDF's thera are text inside the images (to complete the example, this could be a problem, for example, a specific price)
